# LISTA 7 - OBTENCIÓN DE DATOS

## <ins>Ejercicios Obligatorios </ins>

## EJERCICIO 1

In [85]:
import codecs
from datetime import date
from datetime import datetime
from faker import Faker
import pymysql
import random
import requests
import csv
import pandas as pd
from sqlalchemy import create_engine

NUMERO_CLIENTES = 500
NUMERO_PROVEEDORES = 10
SEMILLA_ALEATORIA_GENERADOR = 10
SEMILLA_ALEATORIA_RANDOM = 1

Faker.seed(SEMILLA_ALEATORIA_GENERADOR)
random.seed(SEMILLA_ALEATORIA_RANDOM)
fake = Faker(['es_ES'])

def build_providers_dataset(number):
  providers = []
  for i in range(1, number+1):
    providers.append({
      "provider_id": i,
      "name": fake.company(),
      "email": fake.company_email(),
      "webpage": fake.domain_name()
    })

  return {
      "providers": providers
  }

def build_products_dataset(providers_info):
  products = []
  url = 'https://drive.google.com/uc?export=view&id=1D9MY0au4b7SXwhUdm6TNfsKfYzdkbAh_'
  content = requests.get(url)
  text = codecs.iterdecode(content.iter_lines(), 'utf-8')
  reader = csv.DictReader(text, delimiter=',', quotechar='"')
  for row in reader:
    products.append(row)

  categories = sorted(set([product['category'] for product in products]))
  categories = [{"category_id": i+1, "name": category} for (i, category) in enumerate(categories)]
  categories_by_name = {category["name"]: category["category_id"] for category in categories}
  products = [{"product_id": i+1, 
              "name": product["name"], 
              "price": float(product["price"]), 
              "category_id": categories_by_name[product["category"]],
              "provider_id": random.choice(providers_info)["provider_id"]} 
              for (i, product) in enumerate(products)]
  return {
      'products': products,
      'categories': categories
  }

def build_people_dataset(number):

  people = []
  addresses = []
  payment_info = []
  address_id = 0
  payment_id = 0

  for i in range(1, number+1):
    # Person data
    people.append({
      "person_id": i,
      "first_name": fake.first_name(),
      "last_name": fake.last_name(),
      "birth_date": fake.date_between_dates(datetime(1960, 1, 1), datetime(2002, 6, 1)),
      "email": fake.email(),
      "phone": fake.phone_number(),
      "username": fake.user_name(),
      "password": fake.sha256(),
      "job": fake.job()
    })

    # Payment information
    if random.choice([False]*1 + [True]*2):
      payment_id += 1
      payment_info.append({
          "payment_id": payment_id,
          "person_id": i,
          "expiration": fake.credit_card_expire(),
          "number": fake.credit_card_number(),
          "provider": fake.credit_card_provider(),
          "security_code": fake.credit_card_security_code()
      })

    # Registered addresses
    for j in range(random.choice([1]*43 + [2]*6 + [3])):
      address_id+=1
      addresses.append(
      {
        "address_id": address_id,
        "person_id": i,
        "city": fake.city(),
        "number": fake.building_number(),
        "country": "España",
        "zipcode": fake.postcode(),
        "street": fake.street_name()
      })

  return {
      "people": people,
      "addresses": addresses,
      "payment_information": payment_info,
  }

def build_network_dataset(people_info):

  WEB_PAGES = [fake.uri_path() for i in range(0,100)]
  ACCESS_METHOD_PROPORTION = ['GET'] * 10 + ['POST'] 
  pages = []
  accesses = []
  access_id = 0

  for i in range(0, len(WEB_PAGES)):
    pages.append({
        "page_id": i+1,
        "path": WEB_PAGES[i]
    })

  for person in people_info:
    # Access to webpages
    for j in range(int(random.gauss(60, 40))):
      access_id += 1
      accesses.append({
          "access_id": access_id,
          "person_id": person["person_id"],
          "method": random.choice(ACCESS_METHOD_PROPORTION),
          "ip": fake.ipv4_public(),
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          "page_id": random.randint(1, len(WEB_PAGES)-1)
      })

  # Anonymous access
  for i in range(int(random.gauss(1000, 100))):
    access_id += 1
    accesses.append({
        "access_id": access_id,
        "person_id": None,
        "method": random.choice(ACCESS_METHOD_PROPORTION),
        "ip": fake.ipv4_public(),
        "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
        "page_id": random.randint(1, len(WEB_PAGES)-1)
    })

  return {
    "web_pages":  pages,
    "accesses": accesses
  }

def build_shopping_dataset(people, products, people_addresses):

  shopping_carts = []
  shopping_cart_products = []
  orders = []
  order_products = []
  invoices = []
  cart_id = 0
  shopping_cart_id = 0
  order_id = 0
  order_product_id = 0
  invoice_id = 0

  PRODUCTS_PROBABILITY = [1]*2 + [2] * 3 + [3] * 3 + [4]*2 + [5]
  ORDER_PROBABILITY = [0]+[1]*7+[2]*3+[3]*3+[4]*2+[5]
  QUANTITY_PROBABILITY = [1]*5 +[2]*2 +[3]
  RATING_PROBABILITY = [1]+[2]+[3]*2+[4]*4+[5]*3

  for person in people:
    # Build shopping cart
    if random.choice([False * 9] + [True]):
      cart_id += 1
      shopping_carts.append({
          "cart_id": cart_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
      })

      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        shopping_cart_id += 1
        shopping_cart_products.append({
            "cart_id": cart_id,
            "product_id": product["product_id"],
            "quantity": random.choice(QUANTITY_PROBABILITY)
        })
    
    # Build orders
    for i in range(0, random.choice(ORDER_PROBABILITY)):
      order_id += 1
      order_price = 0
      chosen = random.sample(products, k = random.choice(PRODUCTS_PROBABILITY))
      for product in chosen:
        order_product_id += 1
        quantity = random.choice(QUANTITY_PROBABILITY)
        order_products.append({
            "order_id": order_id,
            "product_id": product["product_id"],
            "quantity": quantity
        })
        order_price += quantity * product['price']

      person_addresses = [address for address in people_addresses if address["person_id"] == person["person_id"]]
      delivery_address = random.choice(person_addresses)
      billing_address = random.choice(person_addresses)
      orders.append({
          "order_id": order_id,
          "person_id": person["person_id"],
          "date": fake.date_time_between(datetime(2020,1,1,0,0,0), datetime(2020,9,1,23,59,59)),
          # Purposely left wrong
          "delivery_address": delivery_address['address_id'],
          "billing_address": billing_address['address_id'],
          "price": order_price
      })

  # Build invoices
  for order in random.choices(orders, k = int(len(orders) * 0.8)):
    invoice_id += 1
    invoices.append({
      "invoice_id": invoice_id,
      "order_id": order["order_id"],
      "date": fake.date_time_between(order["date"], datetime(2020,9,1,23,59,59)),
      "rating": random.choice(RATING_PROBABILITY)
    })

  return {
      'carts': shopping_carts,
      'cart_product': shopping_cart_products,
      'orders': orders,
      'order_product': order_products,
      'invoices': invoices    
  }

dataset = {}
dataset.update(build_providers_dataset(NUMERO_PROVEEDORES))
dataset.update(build_products_dataset(dataset['providers']))
dataset.update(build_people_dataset(NUMERO_CLIENTES))
dataset.update(build_network_dataset(dataset['people']))
dataset.update(build_shopping_dataset(dataset['people'], dataset['products'], dataset['addresses']))
pd.read_sql_query("SELECT * FROM products LIMIT 3", dataset) # A ratos falla el random y sino es que dict no tiene cursor

/tmp/ipykernel_13305/822011591.py:245: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query("SELECT * FROM products LIMIT 3", dataset)


AttributeError: 'dict' object has no attribute 'cursor'

La base de datos shop tiene una tabla people con información sobre los clientes de la tienda ficticia. Escribe el código Python necesario para inicializar un DataFrame con el contenido de la tabla:

## EJERCICIO 2

In [2]:
import pandas as pd
meteo_df = pd.read_csv("https://datosabiertos.carm.es/odata/Agricultura/IMIDA_dia_2018.csv",
                      header=0, sep=";", decimal=",", quotechar="\"", encoding='ISO-8859-1',) # El archivo es UTF-8 pero no me lo reconoce bien, en internet se sugiere usar ISO
meteo_df

,FECHA,EST,MUNICIPIO,PARAJE,TMED,TMAX,TMIN,HRMED,HRMAX,HRMIN,PREC,RADMED,VVMED,VVMAX,DVMED,ETO
0,01/01/18,AL31,Totana,Lebor,12.87,19.17,5.909,46.44,75.90,22.91,0.0,124.03,2.05,6.821,249.66,2.23
1,02/01/18,AL31,Totana,Lebor,13.05,20.99,8.200,44.91,64.69,23.74,0.0,122.80,1.69,8.900,272.61,2.24
2,03/01/18,AL31,Totana,Lebor,13.57,22.57,7.060,61.66,81.00,37.48,0.0,124.63,0.85,3.077,13.25,1.34
3,04/01/18,AL31,Totana,Lebor,14.52,23.84,8.140,63.91,88.60,33.87,0.0,124.25,1.12,4.753,291.87,1.67
4,05/01/18,AL31,Totana,Lebor,10.36,17.70,4.565,77.86,99.90,44.64,0.0,125.46,0.99,3.665,260.43,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17742,27/12/18,TP91,Torre Pacheco,Torre Pacheco,10.76,16.03,7.390,88.28,99.35,65.07,0.2,93.96,0.74,3.626,266.05,0.79
17743,28/12/18,TP91,Torre Pacheco,Torre Pacheco,11.19,15.81,6.600,83.62,96.95,58.85,0.0,93.32,0.66,3.724,320.25,0.80
17744,29/12/18,TP91,Torre Pacheco,Torre Pacheco,12.23,14.68,10.190,91.24,98.00,73.23,9.2,35.73,1.35,5.978,15.21,0.81
17745,30/12/18,TP91,Torre Pacheco,Torre Pacheco,11.01,15.29,6.700,85.61,96.72,62.92,0.2,108.46,1.13,4.802,15.22,0.92


La siguiente URL https://datosabiertos.carm.es/odata/Agricultura/IMIDA_dia_2018.csv contiene el informe meteorológico diario de las diferentes estaciones meteorológicas de la Región de Murcia a lo largo del año 2018. Observa el contenido del fichero csv y a continuación utiliza la función read_csv de pandas sobre esta URL con los  parámetros necesarios (header, sep, decimal, quotechar y encoding).

## EJERCICIO 3

In [11]:
import pandas as pd
covid_df = pd.read_csv("https://cnecovid.isciii.es/covid19/resources/casos_diagnostico_provincia.csv",
                      header=0, sep=",", decimal=".", quotechar="\"", encoding='ISO-8859-1',) # El archivo es UTF-8 pero no me lo reconoce bien, en internet se sugiere usar ISO
print(covid_df.groupby('provincia_iso')['num_casos'].sum().reset_index().rename(columns={'num_casos': 'TOTAL_CASOS'})) # Equivale a SELECT SUM(NUM_CASOS) AS 'TOTAL_CASOS' FROM CSV GROUP_BY PROVINCIA_ISO

   provincia_iso  TOTAL_CASOS
0              A        95133
1             AB        22129
2             AL        32810
3             AV         9576
4              B       346896
5             BA        39881
6             BI        64523
7             BU        26136
8              C        37876
9             CA        47527
10            CC        20478
11            CE         3758
12            CO        35542
13            CR        35943
14            CS        27077
15            CU        15600
16            GC        17267
17            GI        47214
18            GR        57198
19            GU        15579
20             H        18292
21            HU        14862
22             J        34356
23             L        31092
24            LE        27909
25            LO        23899
26            LU         9097
27             M       493480
28            MA        60913
29            ME         5673
30            MU        90861
31            NC        20932
32        

El Instituto de Salud Carlos III ofrece gratuitamente información actualizada sobre la situación del COVID-19 en España. En la siguiente URL podemos obtener un fichero csv actualizado con los casos positivos notificados por las Comunidades Autónomas a nivel provincial: https://cnecovid.isciii.es/covid19/resources/casos_diagnostico_provincia.csv. En el siguiente enlace se nos describe este conjunto de datos y se nos proporcionan otros conjuntos de datos de interés.

Utiliza el método read_csv con los parámetros adecuados para obtener el DataFrame con los casos positivos por provincia. Muestra a continuación su contenido:

## EJERCICIO 4

In [12]:
movies_df = pd.read_json("https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json")
movies_df

,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height
0,After Dark in Central Park,1900,[],[],None,NaN,NaN,NaN,NaN
1,Boarding School Girls' Pajama Parade,1900,[],[],None,NaN,NaN,NaN,NaN
2,Buffalo Bill's Wild West Parad,1900,[],[],None,NaN,NaN,NaN,NaN
3,Caught,1900,[],[],None,NaN,NaN,NaN,NaN
4,Clowns Spinning Hats,1900,[],[Silent],Clowns_Spinning_Hats,Clowns Spinning Hats is a black-and-white sile...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36268,Aquaman and the Lost Kingdom,2023,"[Jason Momoa, Amber Heard, Willem Dafoe, Patri...",[Superhero],Aquaman_and_the_Lost_Kingdom,Aquaman and the Lost Kingdom is an upcoming Am...,https://upload.wikimedia.org/wikipedia/en/thum...,320.0,163.0
36269,Untitled Ghostbusters: Afterlife sequel,2023,"[Mckenna Grace, Carrie Coon, Finn Wolfhard, Pa...","[Comedy, Supernatural]",Untitled_Ghostbusters:_Afterlife_sequel,The untitled Ghostbusters: Afterlife sequel is...,NaN,NaN,NaN
36270,Rebel Moon,2023,"[Sofia Boutella, Charlie Hunnam, Ray Fisher, D...",[Science Fiction],Rebel_Moon,Rebel Moon is an upcoming American epic space ...,NaN,NaN,NaN
36271,Migration,2023,[],[],Migration_(2023_film),This is a list of productions produced by Illu...,NaN,NaN,NaN


La siguiente URL contiene información de películas estadounidenses obtenidas de la Wikipedia en formato JSON: https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json. Utiliza el método `read_json` de `pandas` para cargar su contenido en un DataFrame:

## EJERCICIO 5

In [36]:
import requests
from bs4 import BeautifulSoup

URL = "https://catalogoreina.com/859-grifos-cocina-roca"
r = requests.get(URL) # Obtenemos el HTML en RAW

html_soup = BeautifulSoup(r.text, 'html.parser') # Limpiamos el HTML con el parser adecuado
tags = html_soup.find_all('a', class_='product-name') # Buscamos todos los enlaces que tengan una clase con product-name
for tag in tags: # Imprimimos solo el título de los enlaces dado a que el texto mostrado por <a> es un recorte para que quede bien la página
    print(tag['title'])

Cala grifo de cocina negro mate caño giratorio Roca A5A846ENB0
CALA Grifo diseño para cocina con caño giratorio Roca A5A846EC00
CALA grifo monomando de cocina con caño giratorio en negro Roca A5A836ENB0
CALA Mezclador para cocina con caño giratorio Roca A5A856EC00
CALA Monomando de Cocina Roca A5A836EC00. Ofertas de grifos de cocina
CARMEN Bimando cocina A5A844BC00 Roca
Exclusivo CALA grifo para cocina Negro de Roca A5A856ENB0 calidad
GLERA diseño ergonómico monomando cocina moderno Roca A5A834DC00
GLERA Grifo fregadero extraíble diseño elegante para cocina A5A814DC00 Roca
GLERA Mezclador monomando para cocina con caño giratorio Roca A5A844DC00
GLERA Monomando para cocina con caño giratorio y ducha Roca A5A854DC00
GLERA Pro - Mezclador monomando para cocina con caño giratorio con muelle Roca A5A8A4DC00


Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la siguiente URL https://catalogoreina.com/859-grifos-cocina-roca y recupera el nombre de los artículos mostrados:

## EJERCICIO 6

In [72]:
import requests
from bs4 import BeautifulSoup

URL = "https://db-engines.com/en/ranking"
r = requests.get(URL)
html_soup = BeautifulSoup(r.text, 'html.parser')
table_tr = html_soup.select('table tr') # Saco toda la tabla a lo bruto
top_10 = table_tr[6:16] # No me salía usando nth-child asi que he ido recortando a mano
i = 0 # Contador para el numero de fila que quede bonito
for row in top_10:
    i += 1
    print(i, end=" ")
    print(row.find('a').text.strip().split()[0]) # Saco el <a> del <tr>, le quito todo lo que no sea texto y de ahí solo quiero la primera palabra, tampoco se usar extract

1 Oracle
2 MySQL
3 Microsoft
4 PostgreSQL
5 MongoDB
6 Redis
7 Snowflake
8 Elasticsearch
9 IBM
10 SQLite


Haciendo uso de la librería `requests` y `BeautifulSoup`, accede a la página web del DB-Engines https://db-engines.com/en/ranking y recupera los nombres de las 10 bases de datos más populares:

PISTA: el selector CSS [`nth-child`](https://developer.mozilla.org/es/docs/Web/CSS/:nth-child) puede serte de utilidad.

OTRA PISTA: para evitar que salga el contenido de la etiqueta `info` junto al nombre de la base de datos, haz uso del método [`extract`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract).

## EJERCICIO 7

In [75]:
import requests
from bs4 import BeautifulSoup

URL = "https://catalogoreina.com/nuestras-marcas-muebles-bano/10233-mueble-bano-con-patas-althea-moderno-3-cajones.html"
r = requests.get(URL)
html_soup = BeautifulSoup(r.text, 'html.parser')
name = html_soup.find('h1').text.strip().replace(".","") # Tiene puntos por uso estético que se pueden quitar
print(name)

 Muebles de baño ALTHEA de 3 cajones


Haciendo uso de la librería `requests` y `BeautifulSoup`, recupera el nombre, los acabados, las medidas y el plazo de entrega del siguiente artículo: https://catalogoreina.com/nuestras-marcas-muebles-bano/10233-mueble-bano-con-patas-althea-moderno-3-cajones.html

## <ins>Ejercicios Opcionales </ins>

## EJERCICIO 8

Esta base de datos también contiene una tabla orders con información de la cabecera de los pedidos de la tienda. En esta tabla existe una columna price que almacena el importe total del pedido. Construye a continuación un DataFrame con las cabeceras de pedidos ordenada por importe de manera descendente:

## EJERCICIO 9

La base de datos también contiene las tablas web_pages y accesses con información de las páginas web de la empresa y de los accesos realizados a ellas respectivamente. Ambas tablas pueden relacionarse por el campo page_id. Inicializa un DataFrame que contenga los campos page_id y path de web_pages y el total de accesos realizado sobre cada una de ellas:

## EJERCICIO 10

Crea un diccionario Python que represente una factura en formato JSON:

* Contendrá 4 campos cuyos valores puedes inventarte, pero siguiendo las siguientes indicaciones:
  * **client_id**: identificador de cliente, de tipo cadena.
  * **products**: array con al menos dos productos de tipo objeto. Cada objeto tendrá dos campos:
    * **name**: de tipo cadena.
    * **price**: de tipo numérico.
  * **date**: de tipo cadena.
  * **address**: de tipo objeto, con los siguientes campos:
    * **street**: de tipo objeto, con los siguientes campos:
      * **name**: de tipo cadena.
      * **number**: de tipo numérico.
    * **zipcode**: de tipo numérico.

El diccionario tendrá que tener un formato tal que sea aceptado por el siguiente [validador](https://jsonlint.com/) de contenido JSON.

## EJERCICIO 11

Utilizando la librería PyPDF4, recupera el número de páginas del siguiente PDF:
```
PDF_URL = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_278_COVID-19.pdf'
```

## EJERCICIO 12

Utilizando la librería PyPDF4, recupera la fecha de creación del documento (campo /CreationDate de los metadatos):

## EJERCICIO 13

Extrae la información del pdf en tablas y obten aquella que contiene los Detalles de los quince países con más casos confirmados fuera de Europa.

PISTA: esta tabla es la última del documento PDF. Utiliza la función len para obtener el total de tablas extraídas y saber cuál de ellas seleccionar.

## EJERCICIO 14

Mediante `read_excel`, carga el contenido de todas las hojas del fichero excel "Orders-With Nulls.xlsx", muestra el nombre de todas las hojas y muestra la hoja denominada `Summary`: